In [9]:
import torch
import torch.nn.functional as F
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn
from torch.utils.data import DataLoader
from model import Encoding_Network
from Triplet_Dataset import TripletData

In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
device

device(type='cuda')

In [35]:
class Longer_Encoding_Network(nn.Module):
    
    def __init__(self):
        
        super(Longer_Encoding_Network, self).__init__()
        self.conv = nn.Sequential(nn.Conv2d(7, 32, kernel_size=2), nn.ReLU(), nn.Conv2d(32, 64, kernel_size=2), nn.BatchNorm2d(64), nn.ReLU(),
                                 nn.MaxPool2d(2,2))

        self.linear = nn.Linear(576, 784)
        
        
    def forward(self,x):
        x = self.conv(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.linear(x))
      
        return x
    
class ScoreNetL2(nn.Module):#
    def __init__(self, Encoder_Path):
        super(ScoreNet, self).__init__()
        self.Enc = Longer_Encoding_Network()
        self.Enc.load_state_dict(torch.load(Encoder_Path))
        
        self.linear = nn.Sequential(nn.Linear(1568, 1024), nn.BatchNorm1d(1024), nn.ReLU(), nn.Linear(1024,1))
        
    def forward(self,Anchor, Sample):
        AnchorVec = self.Enc(Anchor)
        SampleVec = self.Enc(Sample)
        x = torch.cat((AnchorVec, SampleVec), 1)
        x = self.linear(x)
        return x

In [36]:
dset =  TripletData('training_neighbors.csv', True)
trainset, testset = dset.split(.95)

trainloader= torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=0 )
testloader= torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=0 )

Train Size: 4758327
Test Size: 250439


In [37]:
def train(model,criterion,optimizer,num_epochs,trainloader,testloader):
    
    for epoch in range(num_epochs):
        running_loss=0.0
        
        for i , (label, anchor, sample) in enumerate(trainloader):
            anchor = anchor.to(device)
            sample = sample.to(device)
            label = label.to(device)
            label = label.unsqueeze(1)
            label = label.to(torch.float32)
            
           
            #forward prop
            
            score = model(anchor,sample)
            
            loss = criterion(score, label)

            #backprop
            optimizer.zero_grad()
            loss.backward()

            #take step
            optimizer.step()
            losss= loss.item()
            
            running_loss+=losss
            
            
           
            if i %500==499:
                print('[%d,%5d] loss: %.3f' % (epoch+1 , i+1, running_loss/500))
                running_loss=0.0
            if i % 2000 == 1999: 
                print("Train accuracy: "+ str(check_accuracy(trainloader, model)*100))
                print("Test accuracy: "+ str(check_accuracy(testloader, model)*100))

In [38]:
def check_accuracy(loader, model, full=False):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for i , (label, anchor, sample) in enumerate(trainloader):
            if i > 200:
                model.train()
                return num_correct/num_samples
                
            anchor = anchor.to(device)
            sample = sample.to(device)
            label = label.to(device)
            label = label.unsqueeze(1)
            label = label.to(torch.float32)
            
           
            #forward prop
            
            score = model(anchor,sample)
            
            
            predictions = torch.sigmoid(score) > .5
            
            
            num_correct += (predictions == label).sum()
            
         
            
            
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples

In [39]:
model = ScoreNet('./previous_models/LVec2.pth').to(device)

In [44]:
batch_size = 16
learning_rate = .5e-3
num_epochs = 10
criterion3 = nn.BCEWithLogitsLoss()
optimizer3=optim.Adam(model.parameters(),lr=learning_rate)
#optimizer3 = optim.SGD(my_model.parameters(),lr=learning_rate,momentum=.9)
num_epochs=15
train(model,criterion3,optimizer3,num_epochs,trainloader,testloader)

[1,  500] loss: 0.346
[1, 1000] loss: 0.357
[1, 1500] loss: 0.343
[1, 2000] loss: 0.358
Train accuracy: tensor(84.4216, device='cuda:0')
Test accuracy: tensor(83.6132, device='cuda:0')
[1, 2500] loss: 0.391
[1, 3000] loss: 0.387
[1, 3500] loss: 0.380
[1, 4000] loss: 0.372
Train accuracy: tensor(83.1468, device='cuda:0')
Test accuracy: tensor(82.4627, device='cuda:0')
[1, 4500] loss: 0.378
[1, 5000] loss: 0.384
[1, 5500] loss: 0.374
[1, 6000] loss: 0.384
Train accuracy: tensor(83.5199, device='cuda:0')
Test accuracy: tensor(84.1729, device='cuda:0')
[1, 6500] loss: 0.372
[1, 7000] loss: 0.376
[1, 7500] loss: 0.381
[1, 8000] loss: 0.381
Train accuracy: tensor(83.9552, device='cuda:0')
Test accuracy: tensor(83.8619, device='cuda:0')
[1, 8500] loss: 0.377
[1, 9000] loss: 0.382
[1, 9500] loss: 0.386
[1,10000] loss: 0.385
Train accuracy: tensor(84.0796, device='cuda:0')
Test accuracy: tensor(83.8308, device='cuda:0')
[1,10500] loss: 0.378
[1,11000] loss: 0.380
[1,11500] loss: 0.392
[1,12000]

[1,90000] loss: 0.373
Train accuracy: tensor(82.6492, device='cuda:0')
Test accuracy: tensor(83.9241, device='cuda:0')
[1,90500] loss: 0.378
[1,91000] loss: 0.381
[1,91500] loss: 0.376
[1,92000] loss: 0.385
Train accuracy: tensor(82.6803, device='cuda:0')
Test accuracy: tensor(82.4627, device='cuda:0')
[1,92500] loss: 0.373
[1,93000] loss: 0.375
[1,93500] loss: 0.382
[1,94000] loss: 0.376
Train accuracy: tensor(84.0174, device='cuda:0')
Test accuracy: tensor(83.0224, device='cuda:0')
[1,94500] loss: 0.377
[1,95000] loss: 0.383
[1,95500] loss: 0.367
[1,96000] loss: 0.378
Train accuracy: tensor(83.6754, device='cuda:0')
Test accuracy: tensor(82.9602, device='cuda:0')
[1,96500] loss: 0.378
[1,97000] loss: 0.364
[1,97500] loss: 0.381
[1,98000] loss: 0.383
Train accuracy: tensor(83.8308, device='cuda:0')
Test accuracy: tensor(83.6132, device='cuda:0')
[1,98500] loss: 0.387
[1,99000] loss: 0.384
[1,99500] loss: 0.368
[1,100000] loss: 0.372
Train accuracy: tensor(84.5460, device='cuda:0')
Tes

KeyboardInterrupt: 

In [45]:
torch.save(model.state_dict(), './previous_models/ScoreNetL2-84.pth')